# DNA Panda (Multiuser)

This is a small script that will import your genome, and query specified genes against NCBI returning a data_frame and .csv with positive matches. 

In [5]:
# Imports
import os
import pandas as pd
from os import listdir
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pylev

import re

import seaborn as sns
sns.set_style('darkgrid')
sns.color_palette('Spectral')
import matplotlib.pyplot as plt


import requests

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

## Import User Data

In [6]:
import glob
# Step 1: get a list of all csv files in target directory
my_dir = "/home/mark/coursework/DNA/data"
filelist = []
filesList = []
user_frame = []
os.chdir( my_dir )

# Step 2: Build up list of files:
for files in glob.glob("*.txt"):
    fileName, fileExtension = os.path.splitext(files)
    filelist.append(fileName) #filename without extension
    filesList.append(files) #filename with extension
    
    

# Step 3: Build up DataFrame:
df = pd.DataFrame()


dfs = [pd.read_table(ijk, sep='\t', 
                   dtype={'rsid':'str', 'chromosome':'object', 'position':'int', 'genotype':'str'}, 
                   comment='#').assign(ID=ijk) \
       for ijk in filesList]

df = pd.concat(dfs, ignore_index=True, axis=0)

In [7]:
# Read the data into a pandas DataFrame and do some EDA
import_frame = pd.read_csv("/home/mark/coursework/DNA/rccx.csv") 
merged_frame = pd.concat([data_frame, import_frame], axis=0, sort=True)
print(merged_frame.head())
df = pd.DataFrame(merged_frame)
df.head().fillna("0")

ParserError: Error tokenizing data. C error: Expected 9 fields in line 601948, saw 12


In [ ]:
#df.isna().any()
# How many chromosomes are on the Y chromosome?
df['chromosome'].unique()
Y_chromosome = df[df.chromosome == 'Y']
len(Y_chromosome)
# Show unique counts
df.nunique()


In [ ]:
## Display how many missing SNPs are in your genome
genotype_na = df[df.genotype == '--']
len(genotype_na)

In [ ]:
# Print the length of any chromosome
df6 = df[df.chromosome == "6"]
len(df6)

In [ ]:
df6.head()

In [ ]:
# See the frequency of genotypes
df6['genotype'].value_counts()

## Isolate the RCCX module

In [ ]:
# CYP21A2 :: 32,038,306 to 32,041,670 on chromosome 6 

# tnxb :: 32,041,153 to 32,109,338 

# C4 :: 31,982,057 to 32,002,681

# stk19 :: 31,971,175 to 31,981,446 

In [ ]:
rccx = df6[(df6['position'] >= 31971175) & (df6['position'] <= 32109338)]
rccx = rccx[rccx.genotype != "--"]
rccx.count()

In [ ]:
rccx['genotype'].value_counts()

In [ ]:
pd.options.display.max_rows = 999
rccx
rccx['Parsed'] = "0"

## Crawling NCBI

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
count = 0
for i, row in rccx.iterrows():
    count = count + 1
    if(row.Parsed != "1"):
        try:
            print("trying...", row.rsid,"(", count, "out of", len(rccx['rsid']),")")
            url = "https://www.ncbi.nlm.nih.gov/snp/" + row.rsid + "#clinical_significance"
            response = urllib.request.urlopen(url)
            html = response.read()
            bs = BeautifulSoup(html, "html.parser")

            classification = bs.find(id="clinical_significance")

            if classification:
                rows = classification.find_all("tr")
                ClinVar = []
                for row in rows:
                    cols = row.find_all("td")
                    cols = [ele.text.strip() for ele in cols]
                    ClinVar.append([ele for ele in cols if ele])
                listToStr = ' '.join([str(elem) for elem in ClinVar]) 

                rccx.at[i, 'ClinVar'] = listToStr

            ncbi = bs.find(class_="summary-box usa-grid-full")
            if ncbi:
                dbSNP = []

                rows = ncbi.find_all("div")

                for row in rows:
                    cols = row.find_all("div")
                    cols = [ele.text.strip() for ele in cols]
                    dbSNP.append(cols)

                try:
                    print("Risk", dbSNP[2][0][0])
                    print("Frequency",dbSNP[2][0][3:7])

                    rccx.at[i, 'Risk'] = dbSNP[2][0][0]
                    rccx.at[i, 'Frequency'] = dbSNP[2][0][3:7]
                except IndexError:
                    print("index error")

                dbSNPTwo= []
                rows = ncbi.find_all("dl")

                for row in rows:
                    cols = row.find_all("dd")
                    cols = [ele.text.strip() for ele in cols]
                    dbSNPTwo.append(cols)

            try:
                print("Gene", dbSNPTwo[1][1].split(' ')[0])
                rccx.at[i, 'Gene'] = dbSNPTwo[1][1].split(' ')[0]
                print("Publications", dbSNPTwo[1][2][0]) 
                rccx.at[i, 'Citations'] = dbSNPTwo[1][2][0]
                rccx.at[i, 'Parsed'] = "1"
            except IndexError:
                    print("index error")


        except urllib.error.HTTPError:
            print(url + " was not found or on dbSNP or contained no valid information")


In [ ]:
rccx

In [ ]:
rccx_filled = rccx.fillna("0")

In [ ]:
rccx_filled

In [ ]:
rccx_present = rccx_filled
rccx_present = rccx_filled[rccx_filled.apply(lambda x: x.Risk in x.genotype, axis=1)]

In [ ]:
rccx_present


In [ ]:
# Correlation Matrix 
f = plt.figure(figsize=(19, 15))
plt.matshow(rccx_present.corr(), fignum=f.number)
plt.xticks(range(rccx_present.shape[1]), rccx_present.columns, fontsize=10, rotation=45)
plt.yticks(range(rccx_present.shape[1]), rccx_present.columns, fontsize=10)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=12)
plt.title('Correlation Matrix', fontsize=14); 

In [ ]:
sns.heatmap(rccx, annot=True)

In [ ]:
#rccx.to_csv('rccx.csv', index=False)
with open('rccx.csv', 'a') as f:
    rccx.to_csv(f, header=False)